<h3> Installing java

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

<h3> Installing spark nlp and pyspark libraries

In [0]:
!pip install spark-nlp==2.6.0
!pip install pyspark==2.4.4

In [0]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np
from numpy import *
import sparknlp
spark = sparknlp.start()
import os
import tweepy as tw
from array import array
import time

In [0]:
print("Spark NLP version", sparknlp.version()) 
print("Apache Spark version:", spark.version)

Spark NLP version 2.6.0
Apache Spark version: 2.4.5

<h3> Reading twitter data file

In [0]:
trainDataset = spark.read.csv('/FileStore/Data/cleaned_200k.csv',header=True)
      
trainDataset.show(10, truncate=False)

+---+-------------------------------------------------------------------------------------------------------------------+--------+
_c0|description |category|
+---+-------------------------------------------------------------------------------------------------------------------+--------+
0 |spring break in plain city it s snowing |0 |
1 |noooo it s raining today and we have to walk around arlington national cemetary |0 |
2 |missing my italian chef |0 |
3 |nawong how do i go about getting access to idzr org i m dying to try it out |0 |
4 |nooooooooo this is largely private now right |0 |
5 |talking to designers i can t squeeze any design juice from my brain for my personal website |0 |
6 |is glad she only has to get her kids up more times before summer vacation time to get ready for work |0 |
7 |i have a friend who is still convinced she wasn t raped by her boyfriend she said no but they were dating after all|0 |
8 |wondering why things aren t going to plan like they should be |0 |
9 |me neither and i m super dupe busy got vms smh i gotta go through and return calls i m tired dude |0 |
+---+-------------------------------------------------------------------------------------------------------------------+--------+
only showing top 10 rows

In [0]:
from pyspark.sql.functions import col
trainDataset.groupBy("category").count().orderBy(col("count").desc()).show()

+--------+-----+
category|count|
+--------+-----+
 0|95722|
 4|95572|
+--------+-----+

<h4> Pre-processing - creating tokenizer, normalizer and stopWordsCleaner

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("description") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")
 
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

<h3> Creating bert embeddings

In [0]:
bert_embeddings = BertEmbeddings\
 .pretrained('bert_base_cased', 'en') \
 .setInputCols(["document",'cleanTokens'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)
 
embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "bert"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")
    
embeddings_finisher = EmbeddingsFinisher() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCols(["finished_sentence_embeddings"]) \
    .setOutputAsVector(True)\
    .setCleanAnnotations(False)

bert_base_cased download started this may take some time.
Approximate size to download 389.2 MB
[ | ][OK!]

<h3> Pipelining the data

In [0]:
from pyspark.ml.feature import CountVectorizer, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler, SQLTransformer
 
label_stringIdx = StringIndexer(inputCol = "category", outputCol = "label")
 
 
nlp_pipeline_bert = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            bert_embeddings,
            embeddingsSentence,
            embeddings_finisher,
           label_stringIdx])

<h3> Training bert data

In [0]:
nlp_model_bert = nlp_pipeline_bert.fit(trainDataset)
processed_bert = nlp_model_bert.transform(trainDataset)

<h3> Parameter tuning

In [0]:
from pyspark.sql.functions import explode,length
from pyspark.sql.functions import udf

@udf("long")
def num_nonzeros(v):
    return v.numNonzeros()


processed_bert= processed_bert.withColumn("features", explode(processed_bert.finished_sentence_embeddings))
processed_bert_2 = processed_bert.select('description','features','label').where(num_nonzeros("features") == 768)
processed_bert_2.show(5)

+--------------------+--------------------+-----+
 description| features|label|
+--------------------+--------------------+-----+
spring break in p...|[-0.5168011784553...| 0.0|
noooo it s rainin...|[-0.4095828533172...| 0.0|
missing my italia...|[-0.4728558361530...| 0.0|
nawong how do i g...|[-0.0366271995007...| 0.0|
nooooooooo this i...|[-0.0708858519792...| 0.0|
+--------------------+--------------------+-----+
only showing top 5 rows

<h3> Splitting data

In [0]:
(trainingData, testData) = processed_bert_2.select('features','label').randomSplit([0.75, 0.25], seed = 100)

<h3> Creating Logistic Regression model

In [0]:
from pyspark.ml.classification import LogisticRegression
 
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
 
lrModel = lr.fit(trainingData)

<h3> Finding prediction

In [0]:
predictions = lrModel.transform(testData)
 
predictions.select("features","probability","label","prediction")\
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+------------------------------+-----+----------+
 features| probability|label|prediction|
+------------------------------+------------------------------+-----+----------+
[-1.0420821905136108,0.6034...|[0.9970735441878004,0.00292...| 0.0| 0.0|
[-1.199479103088379,0.35205...|[0.9969420142797704,0.00305...| 0.0| 0.0|
[-1.199479103088379,0.35205...|[0.9969420142797704,0.00305...| 0.0| 0.0|
[-1.1088402271270752,0.6440...|[0.9968244511942448,0.00317...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
[-1.2367489337921143,0.2073...|[0.9968129856181948,0.00318...| 0.0| 0.0|
+------------------------------+------------------------------+-----+----------+
only showing top 10 rows

<h3> Finding accuracy of ML model

In [0]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

df = predictions.select("probability","label","prediction").toPandas()

print(classification_report(df.label, df.prediction))
print(accuracy_score(df.label, df.prediction))

/databricks/spark/python/pyspark/sql/dataframe.py:2165: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
 Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
 warnings.warn(msg)
 precision recall f1-score support

 0.0 0.74 0.73 0.73 23970
 1.0 0.73 0.74 0.73 23819

 micro avg 0.73 0.73 0.73 47789
 macro avg 0.73 0.73 0.73 47789
weighted avg 0.73 0.73 0.73 47789

0.7331394253907803

<h3> Saving the model

In [0]:
'''
#lrModel.write().save('dbfs:/saveModel')
import tempfile
path = tempfile.mkdtemp()
lrModel.save(sc, path)
'''

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-3511441527077663> in <module> 
 2 import tempfile
 3 path = tempfile . mkdtemp ( ) 
 ----> 4 lrModel . save ( sc , path ) 

 TypeError : save() takes 2 positional arguments but 3 were given

In [0]:
'''
%py
basePath = "/tmp/sa"
#model.save(basePath + "/lrModel")

# You may also specify "overwrite" just as when saving Datasets and DataFrames:
#lrModel.write().save(basePath)
lrModel.write().overwrite().save(basePath)
'''

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3511441527077662> in <module> 
 4 # You may also specify "overwrite" just as when saving Datasets and DataFrames: 
 5 #lrModel.write().save(basePath) 
 ----> 6 lrModel . write ( ) . overwrite ( ) . save ( basePath ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 197 if not isinstance ( path , basestring ) : 
 198 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 199 self . _jwrite . save ( path ) 
 200 
 201 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o6257.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:1032)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply$mcV$sp(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$1.apply(PairRDDFunctions.scala:958)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:392)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopFile(PairRDDFunctions.scala:957)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply$mcV$sp(RDD.scala:1562)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply(RDD.scala:1541)
	at org.apache.spark.rdd.RDD$$anonfun$saveAsTextFile$1.apply(RDD.scala:1541)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.R

In [0]:
'''
%sh 
rm -rf /tmp/mleap_python_model_export
mkdir /tmp/mleap_python_model_export'''

In [0]:
'''
import mleap.pyspark
from mleap.pyspark.spark_support import SimpleSparkSerializer

lrModel.serializeToBundle("jar:file:/tmp/mleap_python_model_download/sentiment_analysis_pipeline-json.zip", predictions)
'''

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-104790036667614> in <module> 
 2 from mleap . pyspark . spark_support import SimpleSparkSerializer
 3 
 ----> 4 lrModel . serializeToBundle ( "jar:file:/tmp/mleap_python_model_download/sentiment_analysis_pipeline-json.zip" , predictions ) 

 /databricks/python/lib/python3.7/site-packages/mleap/pyspark/spark_support.py in serializeToBundle (self, path, dataset) 
 23 def serializeToBundle ( self , path , dataset = None ) : 
 24 serializer = SimpleSparkSerializer ( ) 
 ---> 25 serializer . serializeToBundle ( self , path , dataset = dataset ) 
 26 
 27 

 /databricks/python/lib/python3.7/site-packages/mleap/pyspark/spark_support.py in serializeToBundle (self, transformer, path, dataset) 
 40 
 41 def serializeToBundle ( self , transformer , path , dataset ) : 
 ---> 42 self . _java_obj . serializeToBundle ( transformer . _to_java ( ) , path , dataset . _jdf ) 
 43 
 44 def deserializeFromBundle ( self , path ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o4601.serializeToBundle.
: java.nio.file.NoSuchFileException: /tmp/mleap_python_model_download/sentiment_analysis_pipeline-json.zip
	at sun.nio.fs.UnixException.translateToIOException(UnixException.java:86)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:102)
	at sun.nio.fs.UnixException.rethrowAsIOException(UnixException.java:107)
	at sun.nio.fs.UnixFileSystemProvider.newByteChannel(UnixFileSystemProvider.java:214)
	at java.nio.file.spi.FileSystemProvider.newOutputStream(FileSystemProvider.java:434)
	at java.nio.file.Files.newOutputStream(Files.java:216)
	at com.sun.nio.zipfs.ZipFileSystem.<init>(ZipFileSystem.java:116)
	at com.sun.nio.zipfs.ZipFileSystemProvider.newFileSystem(ZipFileSystemProvider.java:117)
	at java.nio.file.FileSystems.newFileSystem(FileSystems.java:326)
	at java.nio.file.FileSystems.newFileSystem(FileSystems.java:276)
	at ml.combust.bundle.BundleFile$.apply(BundleFile.scala:70)
	at ml.combust.bundle.BundleFile$.apply(BundleFile.scala:40)
	at ml.combust.mleap.spark.SimpleSparkSerializer$$anonfun$serializeToBundleWithFormat$1.apply(SimpleSparkSerializer.scala:25)
	at ml.combust.mleap.spark.SimpleSparkSerializer$$anonfun$serializeToBundleWithFormat$1.apply(SimpleSparkSerializer.scala:25)
	at resource.DefaultManagedResource.open(AbstractManagedResource.scala:110)
	at resource.AbstractManagedResource.acquireFor(AbstractManagedResource.scala:87)
	at resource.ManagedResourceOperations$class.apply(ManagedResourceOperations.scala:26)
	at resource.AbstractManagedResource.apply(AbstractManagedResource.scala:50)
	at resource.DeferredExtractableManagedResource$$anonfun$tried$1.apply(AbstractManagedResource.scala:33)
	at scala.util.Try$.apply(Try.scala:192)
	at resource.DeferredExtractableManagedResource.tried(AbstractManagedResource.scala:33)
	at ml.combust.mleap.spark.SimpleSparkSerializer.serializeToBundleWithFormat(SimpleSparkSerializer.scala:27)
	at ml.combust.mleap.spark.SimpleSparkSerializer.serializeToBundle(SimpleSparkSerializer.scala:17)
	at sun.reflect.NativeMethodAccessorI

In [0]:
'''
%sh 
#rm -rf /tmp/mleap_python_model_export
'''

rm: cannot remove '/FileStore/tables/': Not a directory

In [0]:
'''
#dbutils.fs.rm("/FileStore/tables", True)
dbutils.fs.cp("file:/tmp/mleap_python_model_export/sentiment_analysis_pipeline-json.zip", "dbfs:/FileStore/sentiment_analysis_pipeline-json.zip")
display(dbutils.fs.ls("file:/tmp/mleap_python_model_export"))
'''

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-104790036667616> in <module> 
 1 #dbutils.fs.rm("/FileStore/tables", True) 
 ----> 2 dbutils . fs . cp ( "file:/tmp/mleap_python_model_export/sentiment_analysis_pipeline-json.zip" , "dbfs:/FileStore/sentiment_analysis_pipeline-json.zip" ) 
 3 display ( dbutils . fs . ls ( "file:/tmp/mleap_python_model_export" ) ) 

 /local_disk0/tmp/1607803526392-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: File file:/tmp/mleap_python_model_export/sentiment_analysis_pipeline-json.zip does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:421)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1$$anonfun$apply$3.apply(DBUtilsCore.scala:114)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1$$anonfun$apply$3.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cp(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils.cp(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
'''
from pyspark.ml import PipelineModel
deserializedPipeline = PipelineModel.deserializeFromBundle("jar:file:/tmp/mleap_python_model_export/sentiment_analysis_pipeline-json.zip")
'''

In [0]:
'''
# test_df = spark.read.parquet("/databricks-datasets/news20.binary/data-001/test").select("text", "topic")
# test_df.cache()
# display(test_df)
testData.show(5)
'''

<h3> Streaming Twitter data

<h4> Getting developer access keys and setting connection

In [0]:
consumer_key= ''
consumer_secret= ''    #INPUT TWITTER API KEY
access_token= ''
access_token_secret= ''

In [0]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

<h4> Filter tweets for covid data

In [0]:
search_words = "FDA+covid+vaccine -filter:retweets"
date_since = "2020-12-11"
date_until = "2020-12-12"

<h4> Collect tweets

In [0]:
tweets = tw.Cursor(api.search,
              q=search_words,
              tweet_mode='extended',
              geocode="42.3601,-71.0589,5mi",
              lang="en",
              since=date_since,
              until=date_until).items(5000)
tweets

Out[172]: <tweepy.cursor.ItemIterator at 0x7fb9b0da0160>

In [0]:
users_locs = [[tweet.user.screen_name, tweet.user.location, tweet.full_text] for tweet in tweets]
dataset = pd.DataFrame(data=users_locs, 
                    columns=['user', "location", "description"])
dataset.head()

,user,location,description
0,ethicalpsycholo,"Boston, MA",Is Trump deliberately politicizing vaccine app...
1,BU_Law,"Boston, MA",Confused about the FDA approval process for fo...
2,ragoninstitute,"Cambridge, MA",“The process lasts in the body for about 36 ho...
3,CecilWebsterMD,Boston,Looking forward to FDA officials not being ant...
4,BenjaminConteh9,"Medford, MA","Would you buy:\n1. A product that is so good, ..."


In [0]:
len(dataset)

Out[174]: 53

<h4>Cleaning live twitter data

In [0]:
import re
corpus = []
for i in range(len(dataset)):     #  for i in range(0, 1000):
  text = dataset['description'][i]
  #text = text.lower()
  text = re.sub('(@\w+)', '', text)  # Removing @ followed by words i.e. usernames
  text = re.sub('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)   # Removing hyperlinks
  text = re.sub('\s+', ' ', text) # Removing multiple spaces with single space
  text = text.strip()
  dataset['description'][i] = text 
 # print('The current sample is: ', i)
print('COMPLETED')

<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \s
<>:6: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \w
<>:8: DeprecationWarning: invalid escape sequence \s
<command-104790036667638>:6: DeprecationWarning: invalid escape sequence \w
 text = re.sub('(@\w+)', '', text) # Removing @ followed by words i.e. usernames
<command-104790036667638>:7: DeprecationWarning: invalid escape sequence \w
 text = re.sub('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text) # Removing hyperlinks
<command-104790036667638>:8: DeprecationWarning: invalid escape sequence \s
 text = re.sub('\s+', ' ', text) # Removing multiple spaces with single space
The current sample is: 0
The current sample is: 1
The current sample is: 2
The current sample is: 3
The current sample is: 4
The current sample is: 5
The current sample is: 6
The current sample is: 7
The current sample is: 8
The current sample is: 9
The current sample is: 10
The current sample is: 11
The current sample is: 12
The current sample is: 13
The current sample is: 14
The current sample is: 15
The current sample is: 16
The current sample is: 17
The current sample is: 18
The current sample is: 19
The current sample is: 20
The current sample is: 21
The current sample is: 22
The current sample is: 23
The current sample is: 24
The current sample is: 25
The current sample is: 26
The current sample is: 27
The current sample is: 28
The current sample is: 29
The current sample is: 30
The current sample is: 31
The current sample is: 32
The current sample is: 33
The current sample is: 34
The current sample is: 35
The current sample is: 36
The current sample is: 37
The current sample is: 38
The current sample is: 39
The current sample is: 40
The current sample is: 41
The current sample is: 42
The current sample is: 43
The current sample is: 44
The current sample is: 45
The current sample is: 46
The current sample is: 47
The current sample is: 48
The current sample is: 49
The current sample is: 50
The current sample is: 51
The current sample is: 52
COMPLETED

In [0]:
spark_df = spark.createDataFrame(dataset)
spark_df.show(5)

+---------------+-------------+--------------------+
 user| location| description|
+---------------+-------------+--------------------+
ethicalpsycholo| Boston, MA|Is Trump delibera...|
 BU_Law| Boston, MA|Confused about th...|
 ragoninstitute|Cambridge, MA|“The process last...|
 CecilWebsterMD| Boston|Looking forward t...|
BenjaminConteh9| Medford, MA|Would you buy: 1....|
+---------------+-------------+--------------------+
only showing top 5 rows

<h4> Encoding bert embeddings

In [0]:

processed_bert_new_tweet = nlp_model_bert.transform(spark_df)

In [0]:
processed_bert_new_tweet= processed_bert_new_tweet.withColumn("features", explode(processed_bert_new_tweet.finished_sentence_embeddings))
processed_bert_new_tweet_2 = processed_bert_new_tweet.select('description','features').where(num_nonzeros("features") == 768)
processed_bert_new_tweet_2.show(5)

+--------------------+--------------------+
 description| features|
+--------------------+--------------------+
Is Trump delibera...|[0.29425099492073...|
Confused about th...|[0.30024144053459...|
“The process last...|[0.17655995488166...|
Looking forward t...|[-0.1337188482284...|
Would you buy: 1....|[0.21052804589271...|
+--------------------+--------------------+
only showing top 5 rows

In [0]:
predictions_new_tweet = lrModel.transform(processed_bert_new_tweet_2)

In [0]:
predictions_new_tweet.select("description","features","probability","prediction").show(3)

+--------------------+--------------------+--------------------+----------+
 description| features| probability|prediction|
+--------------------+--------------------+--------------------+----------+
Is Trump delibera...|[0.29425099492073...|[0.57100748927619...| 0.0|
Confused about th...|[0.30024144053459...|[0.49693573139070...| 1.0|
“The process last...|[0.17655995488166...|[0.75071493206671...| 0.0|
+--------------------+--------------------+--------------------+----------+
only showing top 3 rows

In [0]:
pandas_df = predictions_new_tweet.select("prediction").toPandas()
pandas_df.head()

,prediction
0,0.0
1,1.0
2,0.0
3,0.0
4,0.0


In [0]:
pandas_df['CLASS'] = pandas_df['prediction'].map({0.0: 'negative', 1.0 : 'positive'})
pie_chart = pandas_df.groupby(['CLASS']).size()

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.pie(pie_chart, labels=['Negative', 'Positive'], colors=['red', 'green'])
image=plt.show()
display(image)

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 18157); nested exception is: 
	com.databricks.api.base.DatabricksServiceException: QUOTA_EXCEEDED: You have exceeded the maximum number of allowed files on Databricks Community Edition. To ensure free access, you are limited to 10000 files and 10 GB of storage in DBFS. Please use dbutils.fs to list and clean up files to restore service. You may have to wait a few minutes after cleaning up the files for the quota to be refreshed. (Files found: 18157)
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:61)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:105)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:32)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1$$anonfun$11.apply(PythonDriverLocal.scala:957)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1$$anonfun$11.apply(PythonDriverLocal.scala:957)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:54)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:957)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:931)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:876)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:931)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:492)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.com$databricks$backend$daemon$driver$PythonDriverLocal$$outputSuccess(PythonDriverLocal.scala:918)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$repl$6.apply(PythonDriverLocal.scala:364)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$repl$6.apply(PythonDriverLocal.scala:351)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:876)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:351)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:396)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:37